In [1]:
using Pkg
Pkg.activate(joinpath(@__DIR__, "../.."))
using Globtim
using DynamicPolynomials, DataFrames
# Constants and Parameters
const n, a, b = 3, 5.12, 1
const scale_factor = a / b   # Scaling factor appears in `main_computation`, maybe it should be a parameter.
const delta, alpha = 0.5, 1 / 10  # Sampling parameters
const tol_l2 = 3e-4            # Define the tolerance for the L2-norm
f = Rastringin # Objective function

  Activating project at `~/Globtim.jl`


Rastringin (generic function with 1 method)

We set the number of samples used to generate the approximant. It is annoying that the error goes up while the degree has increased.

In [2]:
rand_center = [0.0, 0.0, 0,];
d = 6 # initial degree 
SMPL = 20 # Number of samples
TR = test_input(f, 
                dim = n,
                center=rand_center,
                GN=SMPL, 
                sample_range=scale_factor, 
                degree_max =d+4
                )
pol_cheb = Constructor(TR, d, basis=:chebyshev)
pol_lege = Constructor(TR, d, basis=:legendre);

@polyvar(x[1:n]); # Define polynomial ring 

current L2-norm: 0.1479126520306028
current L2-norm: 46.61160232039606


Solve the system of partial derivatives using `Msolve`. 

In [3]:
df_cheb = solve_and_parse(pol_cheb, x, f, TR)


=== Starting MSolve Parser (dimension: 3) ===
Processed 27 points (0.001s)


Row,x1,x2,x3,z
,Float64,Float64,Float64,Float64
1,9.14153e-15,-4.60798,-4.60798,48.0378
2,-4.60798,-4.60798,-4.60798,87.0568
3,4.60798,-4.60798,-4.60798,87.0568
4,9.03326e-15,2.53969e-14,-4.60798,9.01892
5,-4.60798,2.57726e-14,-4.60798,48.0378
6,4.60798,2.56519e-14,-4.60798,48.0378
7,8.40975e-15,4.60798,-4.60798,48.0378
8,-4.60798,4.60798,-4.60798,87.0568
9,4.60798,4.60798,-4.60798,87.0568


In [7]:
using StaticArrays
using GLMakie
include("../trials/Visual.jl")

grid = scale_factor * generate_grid(3, 60)  # 3D grid
values = map(f, grid) # Prepare level set data for specific level

61×61×61 Array{Float64, 3}:
[:, :, 1] =
 56.5042  55.8158  54.6176  53.311   …  53.311   54.6176  55.8158  56.5042
 55.8158  55.1274  53.9292  52.6226     52.6226  53.9292  55.1274  55.8158
 54.6176  53.9292  52.7309  51.4243     51.4243  52.7309  53.9292  54.6176
 53.311   52.6226  51.4243  50.1177     50.1177  51.4243  52.6226  53.311
 52.5571  51.8687  50.6705  49.3639     49.3639  50.6705  51.8687  52.5571
 53.198   52.5096  51.3114  50.0048  …  50.0048  51.3114  52.5096  53.198
 55.9215  55.2331  54.0349  52.7283     52.7283  54.0349  55.2331  55.9215
 60.617   59.9286  58.7304  57.4238     57.4238  58.7304  59.9286  60.617
 65.6821  64.9937  63.7954  62.4888     62.4888  63.7954  64.9937  65.6821
 68.0201  67.3317  66.1335  64.8269     64.8269  66.1335  67.3317  68.0201
  ⋮                                  ⋱                              ⋮
 65.6821  64.9937  63.7954  62.4888     62.4888  63.7954  64.9937  65.6821
 60.617   59.9286  58.7304  57.4238     57.4238  58.7304  59.9286  6

In [11]:
level_set = prepare_level_set_data(grid, values,8.0, tolerance=0.1)

plot_level_set(to_makie_format(level_set))
